In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

In [2]:
# Load dataset
df_train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
df_test = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = df_train.pop('survived') # binary value
y_test = df_test.pop('survived')

print("df_train:", df_train.shape,"\n")
df_train.head()

df_train: (627, 9) 



,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [3]:
# Feature Extraction
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

features = []
for name in CATEGORICAL_COLUMNS:
    # Get list of all unique values from given feature column
    vocabulary = df_train[name].unique()  
    features.append(tf.feature_column.categorical_column_with_vocabulary_list(name, vocabulary))

for name in NUMERIC_COLUMNS:
    features.append(tf.feature_column.numeric_column(name, dtype=tf.float32))

print(len(features))

9


In [4]:
# Input function

# Returns a function object
def make_input_fn(X, y, num_epochs=10, shuffle=True, batch_size=32):
    # Return a batch of the dataset
    def input_function():
        # Create tf.data.Dataset object with data and its label
        dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
        
        # Randomise data
        if shuffle:
            dataset = dataset.shuffle(1000)
            
         # Split into batches of 32 and repeat for number of epochs
        batch = dataset.batch(batch_size).repeat(num_epochs) 
        return batch
    
    return input_function

# Call the input_function that was returned to us to get a dataset object we can feed to the model
train_input_fn = make_input_fn(df_train, y_train)
test_input_fn = make_input_fn(df_test, y_test, num_epochs=1, shuffle=False)

In [8]:
# Create model
model = tf.estimator.LinearClassifier(feature_columns=features)

model.train(train_input_fn)  # train
result = model.evaluate(test_input_fn)  # get model metrics/stats by testing on tetsing data
clear_output()
print("Model accuracy:", result['accuracy'])

Model accuracy: 0.7386364
